In [4]:
!pip install branca

Defaulting to user installation because normal site-packages is not writeable


In [9]:
import folium
import geopandas as gpd
from branca.colormap import linear
from datetime import datetime
from folium.plugins import TimeSliderChoropleth

# Cargar el archivo de datos geoespaciales (GeoJSON) para los países
world_geojson_path = 'datos/ne_110m_admin_0_countries.json'
gdf = gpd.read_file(world_geojson_path)

# Cargar los datos temporales
# Supongamos que tienes un GeoDataFrame con la información temporal llamado 'data'
# Asegúrate de tener una columna 'date' que contenga las fechas y otras columnas con los indicadores
# data = gpd.read_file('path/to/data.geojson')

# Crear un mapa base
m = folium.Map(location=[30, 0], zoom_start=2)

# Crear un control deslizante de tiempo
time_slider = TimeSliderChoropleth(    
    gdf.to_json(),
    styledict={},
    name='Time Slider',
    init_timestamp=0,
).add_to(m)

# Añadir datos temporales al control deslizante
for date, values in zip(data['date'].unique(), data.groupby('date').mean().iterrows()):
    date_str = date.strftime('%Y-%m-%d')
    values = values[1]
    
    # Crear un estilo para el indicador específico
    style_data = {
        'weight': 1,
        'color': 'black',
        'fillColor': 'YlGnBu',
        'fillOpacity': 0.7,
        'line_opacity': 0.2
    }
    
    # Agregar el estilo al control deslizante
    time_slider.add_to(map, date_str, values, geojson_out='data.geojson', style_function=lambda x: style_data)

# Añadir capa de control de estilo
folium.LayerControl().add_to(m)

# Guardar el mapa como un archivo HTML
m.save('time_slider_map.html')

AssertionError: init_timestamp must be in the range [-0, 0) but got 0

In [3]:
print(folium.__version__)

0.15.1


In [27]:
#Importing Libraries
import pandas as pd
import numpy as np
import json
import folium
from folium import GeoJson
from folium.plugins import TimeSliderChoropleth
from branca.colormap import linear

df=pd.DataFrame({'year_mnth':[1999,2000,2001,2002,2004,2007],
                'zipcode':[47010,47010,47010,28019,28019,28019],
                'city':['Valladolid','Valladolid','Valladolid','Madrid','Madrid','Madrid']})

# Cargar el archivo de datos geoespaciales (GeoJSON) para los países
world_geojson_path = 'datos/gadm36_ESP_4.json'
geo_df = gpd.read_file(world_geojson_path)

#Creating Dataframe
df_house_sold = df[['year_mnth', 'city']]
df_house_sold = df_house_sold.groupby(['year_mnth', 'city'])[['city']].agg(house_sold = 
                                                                                 ('city', 'count')).reset_index()
df_house_sold.rename(columns={'city': 'NAME_2'}, inplace=True)
# df_house_sold = geo_df.merge(df_house_sold, on='NAME_2').drop(columns='NAME_2')
df_house_sold = geo_df.merge(df_house_sold, on='NAME_2')
# df_house_sold = df_house_sold.dissolve(by=['NAME_2', 'year_mnth'], aggfunc='sum').reset_index()
df_house_sold = df_house_sold.dissolve(by=['NAME_2'], as_index=False)
df_house_sold['datemodified'] = pd.to_datetime(df_house_sold['year_mnth'])

#Creating Time Index for slider
datetime_index = pd.DatetimeIndex(df_house_sold['datemodified'])
dt_index_epochs = datetime_index.astype(int) // 10**9
dt_index = dt_index_epochs.astype('U10')

#Converting df into dict type
styledata = {}
n_periods = len(dt_index)
n_sample = len(dt_index)
record_index = 0

for record in df_house_sold.index:
    df_house_sold['geometry'].at[record] = df_house_sold['geometry'][record]
    df = pd.DataFrame({
        'color': df_house_sold.house_sold.to_numpy(),
        'opacity': 0*np.random.normal(size= n_periods)},
        index= dt_index)
    df = df.cumsum()
    df.sample(n_sample, replace=False).sort_index()
    df.loc[df.index == dt_index[record_index], 'opacity'] = np.ones(len(df.index[
        df.index == dt_index[record_index]]))
    
    styledata[record_index] = df
    record_index += 1
    
#Converting values into color
max_color, min_color = 0,0

for datemodified, data in styledata.items():
    max_color = max(max_color, data['color'].max())
    min_color = max(min_color, data['color'].min())
    
cmap = linear.BuPu_06.scale(min_color, max_color)

for city, data in styledata.items():
    data['color'] = data['color'].apply(cmap)
    data['opacity'] = (data['opacity'])
    data['opacity'] = (data['opacity'])
    
#Converting into Timeslider dict type requirements
styledict = {
    str(record): data.sort_index().reset_index().to_dict(orient='index') for 
    record, data in styledata.items()
}

df_house_sold['datemodified'] = df_house_sold['datemodified'].astype('str')

#Showing Map
b = folium.Map(location=[df['lat'].median() - 0.2, #0.1,
                         df['long'].median()+ 0.5], #0.8],
#                zoom_start= 9,
               zoom_start= 9,
#                tiles="Cartodb Positron",
               width="%80", height="%80")
slider = TimeSliderChoropleth(df_house_sold.to_json(), 
                              styledict = styledict, 
                              overlay = True).add_to(b)

folium.Choropleth(
    geo_data = df_house_sold.to_json(),
    data = df_house_sold,
    columns = ['datemodified', 'house_sold'],
    key_on = 'feature.id',
    highlight = True,
    fill_color= 'BuPu',
    fill_opacity=0,
    line_opacity=0.0,
    legend_name = 'Number of Houses',
    ).add_to(b)

b

GEOSException: TopologyException: unable to assign free hole to a shell at -4.1706809997558594 40.5755615234375

In [21]:
geo_df.columns.values

array(['_isDeleted', 'GID_0', 'NAME_0', 'GID_1', 'NAME_1', 'GID_2',
       'NAME_2', 'GID_3', 'NAME_3', 'GID_4', 'NAME_4', 'VARNAME_4',
       'TYPE_4', 'ENGTYPE_4', 'CC_4', 'geometry'], dtype=object)

In [22]:
geo_df.iloc[1]

_isDeleted                                                False
GID_0                                                       ESP
NAME_0                                                    Spain
GID_1                                                   ESP.1_1
NAME_1                                               AndalucÃ­a
GID_2                                                 ESP.1.1_1
NAME_2                                                 AlmerÃ­a
GID_3                                               ESP.1.1.1_1
NAME_3                                                n.a. (15)
GID_4                                             ESP.1.1.1.2_1
NAME_4                                                    Albox
VARNAME_4                                                      
TYPE_4                                             Municipality
ENGTYPE_4                                          Municipality
CC_4                                                           
geometry      POLYGON ((-2.1835498809814